In [1]:
import pandas as pd
import numpy as np
import os as os 

### Load data

In [2]:
dfs_train = {}
dfs_test = {}

In [3]:
for root, directory, files in os.walk("data/train_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_train[file[:-4]] = pd.read_csv(f"{root}/{file}")
for root, directory, files in os.walk("data/test_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_test[file[:-4]] = pd.read_csv(f"{root}/{file}")

In [4]:
dfs_train["df_belgium"]

,Unnamed: 0,season,Avg_away_odds,Avg_home_odds,Avg_draw_odds,Avg_bookie_prediction,Var_away_odds,Var_home_odds,Var_draw_odds,MatchTeams,...,HomeWinRatio,HomeLossRatio,HomeDrawRatio,AwayWinRatio,AwayLossRatio,AwayDrawRatio,HomeTeamAvgShotsOnTarget,AwayTeamAvgShotsOnTarget,HomeTeamScoredRatio,AwayTeamScoredRatio
0,300,1.0,4.666667,1.556667,3.533333,H,0.083333,0.000133,0.043333,Beveren_Genk,...,0.343750,0.406250,0.250000,0.264706,0.500000,0.235294,4.756882,4.309364,0.478261,0.319149
1,301,1.0,8.600000,1.233333,4.900000,H,2.680000,0.003333,0.280000,Anderlecht_Antwerp,...,0.735294,0.029412,0.235294,0.323529,0.470588,0.205882,6.264428,4.510234,0.778761,0.457831
2,302,1.0,2.966667,2.050000,3.166667,H,0.043333,0.032500,0.063333,Lierse_Lokeren,...,0.375000,0.437500,0.187500,0.468750,0.250000,0.281250,4.734209,5.034806,0.472527,0.571429
3,303,1.0,5.733333,1.416667,3.833333,H,0.463333,0.000833,0.043333,Gent_St Truiden,...,0.454545,0.272727,0.272727,0.264706,0.500000,0.235294,5.121079,4.826950,0.552381,0.448980
4,304,1.0,3.916667,1.706667,3.383333,H,0.270833,0.008133,0.110833,Lommel_Molenbeek,...,0.454545,0.272727,0.272727,0.264706,0.500000,0.235294,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5487,5787,21.0,3.914000,1.870000,3.574000,H,0.010480,0.001150,0.013380,St Truiden_Standard,...,0.307829,0.457295,0.234875,0.492017,0.249637,0.258345,4.455075,5.083699,0.436352,0.600632
5488,5788,21.0,7.970000,1.330000,5.290000,H,0.092000,0.000350,0.045500,Club Brugge_Mechelen,...,0.621145,0.182085,0.196769,0.336798,0.426195,0.237006,5.613602,4.621183,0.673558,0.445467
5489,5789,21.0,1.392000,7.320000,4.752000,A,0.001270,0.130750,0.026770,Anderlecht_Kortrijk,...,0.359036,0.407229,0.233735,0.616519,0.156342,0.227139,4.648310,5.431673,0.486462,0.683400
5490,5790,21.0,11.128000,1.228000,6.252000,H,2.046920,0.000370,0.062520,Gent_Oud-Heverlee Leuven,...,0.471366,0.286344,0.242291,0.262570,0.424581,0.312849,5.128811,4.660707,0.574391,0.441392
